# CNN with Batching

2022-07-14

Zooey Nguyen

Our simple CNN model with batching.

## Setup

In [ ]:
import os
import h5py
import numpy as np
import tensorflow as tf
from DataMaker import *

In [ ]:
LATENT_DIM = 16
IMAGE_SHAPE = (5, 127, 127)
BATCH_SIZE = 128
EPOCHS = 10
GB_LIMIT = 5

## Allocate GPU

Make sure to check others' current usage on Aurora in terminal using `watch nvidia-smi`. Set your `GB_LIMIT` accordingly, in gigabytes. Aurora has 50GB of memory.

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=GB_LIMIT*1000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


## Data processing

In [3]:
args_gen = {'X_key': 'image',
    'y_key': 'specz_redshift',
    'scaler': False,
    'labels_encoding': False,
    'batch_size': 32,
    'mode': 'train'}

start_path = "/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_"
end_path = "_with_metadata_with_scaled.hdf5"
TRAIN_PATH = start_path + "training" + end_path
VAL_PATH = start_path + "validation" + end_path
TEST_PATH = start_path + "testing" + end_path

In [4]:
train_hf = h5py.File(TRAIN_PATH)
print(list(train_hf.keys()))
print(train_hf['image'].shape)

['coord', 'dec', 'g_cmodel_mag', 'g_cmodel_magsigma', 'i_cmodel_mag', 'i_cmodel_magsigma', 'image', 'object_id', 'r_cmodel_mag', 'r_cmodel_magsigma', 'ra', 'scaled_mags', 'skymap_id', 'specz_dec', 'specz_flag_homogeneous', 'specz_mag_i', 'specz_name', 'specz_ra', 'specz_redshift', 'specz_redshift_err', 'y_cmodel_mag', 'y_cmodel_magsigma', 'z_cmodel_mag', 'z_cmodel_magsigma']
(229120, 5, 127, 127)


In [5]:
train_gen = HDF5ImageGenerator(src=TRAIN_PATH, **args_gen)
val_gen = HDF5ImageGenerator(src=VAL_PATH, **args_gen)
test_gen = HDF5ImageGenerator(src=TEST_PATH, **args_gen)

## Simple CNN

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer
from tensorflow.keras.layers import Conv2D, MaxPooling2D

### Define model

In [7]:
model = Sequential()
model.add(InputLayer(input_shape=IMAGE_SHAPE))
model.add(Conv2D(32, kernel_size=(3, 3), activation='tanh', padding='same', data_format='channels_first'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32, kernel_size=(2,2),activation='tanh', padding='same', data_format='channels_first'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32, kernel_size=(3,3),activation='relu', padding='same', data_format='channels_first'))
model.add(Conv2D(32, kernel_size=(2,2),activation='relu', padding='same', data_format='channels_first'))
model.add(Flatten())
model.add(Dense(500, activation="tanh"))
model.add(Dense(100, activation="tanh"))
model.add(Dense(1))

In [8]:
model.compile(optimizer='Adam', loss="mse",metrics=[tf.keras.metrics.MeanAbsoluteError()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 127, 127)      1472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 63, 127)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 63, 127)       2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 31, 127)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 31, 127)       4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 31, 127)       4128      
_________________________________________________________________
flatten (Flatten)            (None, 125984)            0

###  Train model

In [10]:
model.fit(train_gen, epochs=EPOCHS, verbose=1)

Epoch 1/10
7160/7160 [==============================] - 761s 73ms/step - loss: 0.3687 - mean_absolute_error: 0.3860
Epoch 2/10
7160/7160 [==============================] - 6048s 845ms/step - loss: 0.3301 - mean_absolute_error: 0.3757
Epoch 3/10
  39/7160 [..............................] - ETA: 1:26:17 - loss: 0.2923 - mean_absolute_error: 0.3578

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7160/7160 [==============================] - 5639s 788ms/step - loss: 0.3305 - mean_absolute_error: 0.3761
Epoch 5/10
7160/7160 [==============================] - 5546s 775ms/step - loss: 0.3303 - mean_absolute_error: 0.3760
Epoch 6/10
7160/7160 [==============================] - 5444s 760ms/step - loss: 0.3303 - mean_absolute_error: 0.3756
Epoch 7/10
7160/7160 [==============================] - 5591s 781ms/step - loss: 0.3300 - mean_absolute_error: 0.3756
Epoch 8/10
7160/7160 [==============================] - 5487s 766ms/step - loss: 0.3302 - mean_absolute_error: 0.3759
Epoch 9/10
7160/7160 [==============================] - 5472s 764ms/step - loss: 0.3301 - mean_absolute_error: 0.3758
Epoch 10/10
7160/7160 [==============================] - 5817s 812ms/step - loss: 0.3304 - mean_absolute_error: 0.3761


### Save model

In [ ]:
MODEL_PATH = "/data/HSC/HSC_v6/models_zooey"
MODEL_NAME = "CNN"
MODEL_VERSION = "1"

In [ ]:
model_file = MODEL_PATH + '/' + MODEL_NAME +\
    '_v' + MODEL_VERSION +\
    '_batch' + str(BATCH_SIZE) +\
    '_epoch' + str(EPOCHS)
print(model_file)

In [25]:
model.save(model, model_file)

INFO:tensorflow:Assets written to: trained_models/cnn_batch128_epochs10/assets


### Evaluate model

In [27]:
results = model.evaluate(val_gen)

896/896 [==============================] - 58s 65ms/step - loss: 0.3257 - mean_absolute_error: 0.3675


In [57]:
val_preds = model.predict(val_gen)